# Importing Libraries

In [72]:
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag

import contractions # for contractions
from bs4 import BeautifulSoup # for HTML tags

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)

True

---

# Data Overview

In [73]:
df = pd.read_csv("IMDB Dataset.csv")
pd.set_option('display.max_colwidth', None) # to display entire content
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [74]:
pd.reset_option('display.max_colwidth') # resetting the option

In [75]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [77]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [78]:
col = df.columns[0]

df['char_count'] = df[col].str.len()

mean_chars = df['char_count'].mean()
max_chars = df['char_count'].max()
min_chars = df['char_count'].min()

print('----Review lengths----')
print("Mean characters:", mean_chars)
print("Max characters:", max_chars)
print("Min characters:", min_chars)

----Review lengths----
Mean characters: 1309.43102
Max characters: 13704
Min characters: 32


---

# Data Cleaning

### -------------------------------------- Removing Extra Spaces

In [79]:
def remove_extra_spaces(text):
    text = re.sub(r'\s+', ' ', text) # replace multiple spaces with a single space
    return text.strip()

Testing the function

In [80]:
text = """
This   is    a    sample    text.  

It  has multiple      spaces, 
tabs\t, and   newlines.  
"""

remove_extra_spaces(text)

'This is a sample text. It has multiple spaces, tabs , and newlines.'

### -------------------------------------- Removing HTML tags

In [81]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

Testing the function

In [82]:
text ="""
<p>I'm loving this product! It's amazing &lt;3</p>
<div>Check out the <a href="https://example.com">website</a> for more info.</div>
<span>Don't forget to review it!</span>
"""
clean_text = remove_html_tags(text)
clean_text = remove_extra_spaces(clean_text)

print(clean_text)


I'm loving this product! It's amazing <3 Check out the website for more info. Don't forget to review it!


### -------------------------------------- Expanding Contractions

In [83]:
def expand_contractions(text):
    return contractions.fix(text)

Testing the function

In [84]:
text = """
I'm excited about this new movie! It's supposed to be amazing.
Don't worry if you haven't seen the trailer yet — they'll show it again soon.
You're going to love it, and I can't wait to discuss it with you!
"""
clean_text = expand_contractions(text)
clean_text = remove_extra_spaces(clean_text)

print(clean_text)

I am excited about this new movie! It is supposed to be amazing. Do not worry if you have not seen the trailer yet — they will show it again soon. You are going to love it, and I cannot wait to discuss it with you!


### -------------------------------------- To Lower Case

In [85]:
def to_lowercase(text):
    return text.lower()

Testing the function

In [86]:
text = "OMG The Movie Was So Great. Absolutely Stunning PERFORMANCE!!"
print(to_lowercase(text))

omg the movie was so great. absolutely stunning performance!!


### -------------------------------------- Removing Special Characters

In [87]:
def remove_special_characters(text):
    text = re.sub(r'[^a-zA-Z ]',' ', text)
    return text

Testing the function

In [88]:
text = """
Wow!!! This product is awesome :) #excited
Price: $19.99 @amazon
Call me at (123) 456-7890 or email me: test@example.com
"""
clean_text = remove_special_characters(text)
clean_text = remove_extra_spaces(clean_text)

print(clean_text)

Wow This product is awesome excited Price amazon Call me at or email me test example com


### -------------------------------------- Removing Stop Words

In [89]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)

    # preserve common negations
    negations = {"not", "no", "nor", "never"}
    stop_words = stop_words - negations

    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

Testing the function

In [90]:
text = "I do not like this product, but it could have been better."

cleaned_text = remove_stopwords(text)
print(cleaned_text)

I not like product , could better .


### -------------------------------------- Lemmatizing

To handle parts of speech correctly

In [91]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

In [92]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    pos_tags = pos_tag(word_tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized)

Testing the function

In [93]:
text = "The cats are running faster than the dogs. She was going to the market and buying fruits."

lemmatize_text(text)

'The cat be run faster than the dog . She be go to the market and buy fruit .'